In [325]:
from utils import Utils, AOC
from session import SESSION
from collections import defaultdict
from matplotlib import pyplot as plt
import numpy as np
import sortedcontainers
import time
from collections.abc import Iterable

In [2]:
aoc = AOC(session=SESSION)

In [3]:
aoc.verify_session()

True

In [4]:
aoc.get_today_file().analyse().head()

Data correctly downloaded and saved locally for next usage.
71% of data are digits. Analyse as text.
0 empty line(s) found. Analyse as monline data.
===== HEAD (1/1) =====
40541D900AEDC01A88002191FE2F45D1006A2FC2388D278D4653E3910020F2E2F3E24C007ECD7ABA6A200E6E8017F92C934CFA0E5290B569CE0F4BA5180213D963C00DC40010A87905A0900021B0D624C34600906725FFCF597491C6008C01B0004223342488A200F4378C9198401B87311A0C0803E600FC4887F14CC01C8AF16A2010021D1260DC7530042C012957193779F96AD9B36100907A00980021513E3943600043225C1A8EB2C3040043CC3B1802B400D3CA4B8D3292E37C30600B325A541D979606E384B524C06008E802515A638A73A226009CDA5D8026200D473851150401E8BF16E2ACDFB7DCD4F5C02897A5288D299D89CA6AA672AD5118804F592FC5BE8037000042217C64876000874728550D4C0149F29D00524ACCD2566795A0D880432BEAC79995C86483A6F3B9F6833397DEA03E401004F28CD894B9C48A34BC371CF7AA840155E002012E21260923DC4C248035299ECEB0AC4DFC0179B864865CF8802F9A005E264C25372ABAC8DEA706009F005C32B7FCF1BF91CADFF3C6FE4B3FB073005A6F93B633B12E0054A124BEE9C570004B245126F6E1

In [300]:
def hex_to_bin(hexa):
    return bin(int(hexa, 16))[2:].zfill(4*len(hexa))

In [301]:
binary = hex_to_bin(aoc.data[0])

In [302]:
def get_meta(b):
    return int(b[:3], 2), int(b[3:6], 2)

In [303]:
get_meta(binary)

(2, 0)

In [304]:
def read_digit(b): # type 4
    version, id_ = get_meta(b)
    i = 6
    digit = ''
    while len(b) > i:
        digit += b[i+1:i+5]
        i += 5
        if b[i-5] == '0':
            break
    return version, id_, i, int(digit, 2)

In [316]:
def binary_read(b, amount=-1):
    if len(b) < 6 or amount == 0:
        return tuple()

    version, type_id = get_meta(b)

    #print(amount, version, type_id, b, int(b, 2))

    if int(b, 2) == 0:
        return tuple()

    # Digit value
    if type_id == 4:
        _, _, length, value = read_digit(b)
        return (version, type_id, length, value), *binary_read(b[length:], amount=amount-1)

    # Operator
    length_type_id = b[6]

    if length_type_id == '0':
        length = int(b[7:22], 2)
        sub = binary_read(b[22:22+length])
        sub_length = sum(el[2] for el in sub) #  if isinstance(el, Iterable)
        return (version, type_id, sub_length + 22, sub), *binary_read(b[22+length:], amount=amount-1)
    elif length_type_id == '1':
        length = int(b[7:18], 2)
        sub = binary_read(b[18:], amount=length)
        sub_length = sum(el[2] for el in sub) #  if isinstance(el, Iterable)
        return (version, type_id, sub_length +18, sub), *binary_read(b[18+sub_length:], amount=amount-1)

In [317]:
def versions_sum(t):
    s = 0
    for el in t:
        s += el[0]
        if isinstance(el[3], tuple):
            s += versions_sum(el[3])
    return s

In [318]:
res = binary_read(binary)

In [295]:
# > 919

In [319]:
versions_sum(res)

945

In [429]:
def binary_compute(b, amount=-1):
    if len(b) < 6 or amount == 0:
        return tuple()

    version, type_id = get_meta(b)

    if int(b, 2) == 0:
        return tuple()

    # Digit value
    if type_id == 4:
        _, _, length, value = read_digit(b)
        return (version, type_id, length, value), *binary_compute(b[length:], amount=amount-1)

    # Operator
    length_type_id = b[6]

    if length_type_id == '0':
        length = int(b[7:22], 2)
        sub = binary_compute(b[22:22+length])
        sub_length = sum(el[2] for el in sub) + 22
    else:
        length = int(b[7:18], 2)
        sub = binary_compute(b[18:], amount=length)
        sub_length = sum(el[2] for el in sub) + 18

    els = [el[3] for el in sub]

    #print(type_id, *els)

    if type_id == 0:
        result = sum(els)
    elif type_id == 1:
        result = 1
        for el in els:
            result *= el
    elif type_id == 2:
        result = min(els)
    elif type_id == 3:
        result = max(els)
    elif type_id == 5:
        result = int(els[0] > els[1])
    elif type_id == 6:
        result = int(els[0] < els[1])
    elif type_id == 7:
        result = int(els[0] == els[1])
    else:
        result = 0

    return (version, type_id, sub_length, result), *binary_compute(b[sub_length:], amount=amount-1)

In [431]:
ex = hex_to_bin("9C0141080250320F1802104A08")
read = binary_compute(binary)
print(read)

((2, 0, 5405, 10637009915279),)


In [353]:
print(binary_read(ex))

((4, 7, 102, ((2, 0, 40, ((2, 4, 11, 1), (4, 4, 11, 3))), (6, 1, 40, ((0, 4, 11, 2), (2, 4, 11, 2))))),)
